In [2]:
from mf_modules.j_header import j_header
#help(j_header)
header={
    'title':'10_JG_SpecGenerator',
    'des':'''
        Generates template markdown files for spec sections <br>
    ''',
    'job_nm' :'Engineering Standards | Digital Design',
    'job_no' : 'J6667',
    'author' : 'JG',
    'fnm':'Spec300_headings',
    'wdir':'.',
    'chdir_to_wdir':False
}
j_header(**header)

In [7]:

import os
from IPython.display import Markdown
from mf_modules.pydtype_operations import read_yaml, read_txt, iconv_catch_utf8_errors
from mf_modules.pydtype_operations import camel_case_to_words

In [26]:

from docx import Document
from docx.shared import Inches
from IPython.display import FileLink, FileLinks
from IPython.display import display
def iter_headings(paragraphs):
    for paragraph in paragraphs:
        if paragraph.style.name.startswith('Heading'):
            yield paragraph
            
def list_headings(document):
    li=[]
    for heading in iter_headings(document.paragraphs):
        li.append(heading.text)
    return li
def get_headings(fpth):
    document = Document(fpth)
    print('Spec-300 input document:')
    print('-----------------------------------------')
    display(FileLink(fpth))
    print('')
    li=list_headings(document)

    print('1st 10 example headings from the Spec-300')
    print('-----------------------------------------')
    for i in range(0,10):
        print(li[i])
    print('')

    li1=[]
    for l in li:
        li1.append(l.split(' ',1))



    fpth_out=os.path.join(os.path.dirname(fpth),os.path.splitext(os.path.basename(fpth))[0]+'headings.txt')
    with open(fpth_out, 'w') as f:
        for item in li1:
            try:
                f.write("%s" % item[0]+"    "+item[1]+"\n")
            except:
                print('ignored item: '+str(item))
    print('')  
    print('txt file output with listed Spec headings:')
    print('-----------------------------------------')
    iconv_catch_utf8_errors(fpth_out)
    FileLink(fpth_out)
    return fpth_out
    
fpth_100 = r'..\data\raw\Spec100.docx'
fpth_300 = r'..\data\raw\Spec300.docx'

# NOT IN USE
#get_headings(fpth_100)
#print('-------------------------')
#print('-------------------------')
#print('-------------------------')
#get_headings(fpth_300)

In [27]:
import pandas as pd
from mf_modules.excel_in import ExcelIn
#df = pd.read_excel( , sheet_name='classification',)
fpth=r'..\DocumentNumberGenerator.xlsm'
xl=ExcelIn(fpth,sheet_name='classification')
df=xl.pytable
df['spec_heading'] = df['uniclass_classification'] + '-' + df['classification_des']
df_filt = df.dropna(subset=['inMfPackage'])
spec_headings = df_filt['spec_heading'].dropna().tolist()

In [28]:
#spec_headings = [s.replace(' ','') for s in spec_headings]

In [45]:
def mdli_to_file(fpth_md, li, runiconv=False):
    """
    writes list to markdown file.
    runs iconv to deal with utf-8 errors.
    Args:
        md_fpth (str): filepath of markdown schedule file to create
        li: list of markdown stings

    Returns:
        md_fpth (str): filepath of markdown schedule created
    """
    with open(fpth_md, 'w') as f:
        for l in li:
            if l == '\\newpage':
                f.write("%s\n" % l)x
            else:
                f.write("%s\n\n" % l)
    f.close()
    if runiconv:
        iconv_catch_utf8_errors(fpth_md)
        
    return fpth_md
        
def spec_template(spec_heading,template_di, datadir = '..\\spec'):
    md = []
    for k, v in template_di.items():
        content = ''
        if k == '__system_name__':
            k = spec_heading
        fpth_ind = os.path.join(datadir,spec_heading.replace(' ','')+'_IndustryStandards.csv')
        if os.path.isfile(fpth_ind):
            if k == 'Industry Standards':
                print(fpth_ind)
                content = pd.read_csv(fpth_ind).dropna().to_markdown()
                k = "%s\n\n" % k
        heading = v + k
        md.append(heading + content)
    return md
    
#display(Markdown([m for m in md]))

template_di = read_yaml(r'..\spec\_spec100_template.yaml')['spec100']
spec_md = []
for spec_heading in spec_headings:
    spec_md.append(spec_template(spec_heading,template_di))
    
for s in spec_md:
    fpth_md = '..\\spec\\'+ s[0].replace('# ','').replace(' ','')+'.md'
    mdli_to_file(fpth_md, s, runiconv=False)

..\spec\Ss_50_30_04_00-AboveGroundWasteWaterDrainage_IndustryStandards.csv


In [39]:
from mf_modules.constants import REFDOCX
from mf_modules.file_operations import fpth_chg_extension

fpth_md = '../spec/Ss_50_30_04_00-AboveGroundWasteWaterDrainage.md'
fpth_docx = fpth_chg_extension(fpth_md, new_ext='docx')
fpth_refdocx = REFDOCX

cmd = 'call pandoc -o "{0}" -f markdown -t docx "{1}" --reference-doc={2} --filter=pandoc-docx-pagebreakpy'.format(
    fpth_docx, fpth_md, fpth_refdocx)
os.system(cmd)

0

In [40]:
from mf_modules.file_operations import open_file
open_file(fpth_docx)